In [3]:
import cv2
import numpy as np
img_colour = cv2.imread('./unocards/r6.jpg')   # open the saved image in colour
####
# Add something to detect color
####
HSVmin = np.array([0, 150, 170])
HSVmax = np.array([10, 255, 255])
img_HSV = cv2.cvtColor(img_colour, cv2.COLOR_BGR2HSV)  # Convert to HSV
img_th = cv2.inRange(img_HSV, HSVmin, HSVmax) 
# img = cv2.cvtColor(img_colour, cv2.COLOR_BGR2GRAY)   # convert to B/W

img_sm = cv2.blur(img_HSV, (5, 5))         # smoothing
# thr_value, img_th = cv2.threshold(img_sm, 160, 255, cv2.THRESH_BINARY_INV)   # binarisation
# img_th = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 35, 2)
kernel = np.ones((3, 3), np.uint8)
img_close = cv2.morphologyEx(img_th, cv2.MORPH_CLOSE, kernel)      # morphology correction
img_canny = cv2.Canny(img_close, 100, 200)                          # edge detection
contours, hierarchy = cv2.findContours(img_close, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)   # extract contours on binarised image, not on canny
cv2.drawContours(img_colour, contours, -1, (0, 255, 0), 2)         # paint contours on top of original coloured mage
cv2.imshow('color with contours',img_colour)
cv2.imshow('thresholded', img_th)
cv2.imshow('canny', img_canny)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
imgc = cv2.imread('./unocards/r6.jpg')    # open the saved image in colour 
for i, c in enumerate(contours):         # loop through all the found contours
    print(i, ':', hierarchy[0, i])          # display contour hierarchy
    print('length: ', len(c))               # display numbr of points in contour c
    perimeter = cv2.arcLength(c, True)     # perimeter of contour c (curved length)
    print('perimeter: ', perimeter)               
    epsilon = 5 # 0.01*perimeter    # parameter of polygon approximation: smaller values provide more vertices
    vertex_approx = len(cv2.approxPolyDP(c, epsilon, True))     # approximate with polygon
    print('approx corners: ', vertex_approx, '\n')                    # number of vertices
    cv2.drawContours(imgc, [c], 0, (0, 255, 0), 3)   # paint contour c
    cv2.putText(imgc, str(i), (c[0, 0, 0]+20, c[0, 0, 1]+30), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 255))    # identify contour c
    [x,y,w,h] = cv2.boundingRect(c)
    cv2.rectangle(imgc, (x,y), (x+w,y+h), (255, 0, 0), 2)
cv2.namedWindow('picture', cv2.WINDOW_NORMAL)
cv2.imshow('picture',imgc)
key = cv2.waitKey(0)
cv2.destroyAllWindows()

0 : [ 4 -1  1 -1]
length:  706
perimeter:  783.8721451759338
approx corners:  12 

1 : [-1 -1  2  0]
length:  203
perimeter:  231.5807341337204
approx corners:  7 

2 : [ 3 -1 -1  1]
length:  3
perimeter:  3.414213538169861
approx corners:  1 

3 : [-1  2 -1  1]
length:  23
perimeter:  25.899494767189026
approx corners:  2 

4 : [ 8  0  5 -1]
length:  726
perimeter:  857.7199051380157
approx corners:  16 

5 : [-1 -1  6  4]
length:  476
perimeter:  542.2741661071777
approx corners:  15 

6 : [ 7 -1 -1  5]
length:  36
perimeter:  40.97056245803833
approx corners:  3 

7 : [-1  6 -1  5]
length:  69
perimeter:  77.69848430156708
approx corners:  4 

8 : [-1  4  9 -1]
length:  703
perimeter:  779.6295045614243
approx corners:  10 

9 : [-1 -1 10  8]
length:  211
perimeter:  238.75230705738068
approx corners:  11 

10 : [11 -1 -1  9]
length:  1
perimeter:  0.0
approx corners:  1 

11 : [-1 10 -1  9]
length:  21
perimeter:  23.899494767189026
approx corners:  2 



In [1]:
import cv2
import numpy as np

# Load the image
img = cv2.imread('./unocards/r6.jpg')

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to remove noise
blur = cv2.GaussianBlur(gray, (5, 5), 0)

# Apply Canny edge detection to find edges
edges = cv2.Canny(blur, 50, 150)

# Find contours in the image
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Loop over the contours and find the one that matches an ellipse
for contour in contours:
    # Fit an ellipse to the contour
    ellipse = cv2.fitEllipse(contour)
    (x, y), (a, b), angle = ellipse
    
    # Calculate the aspect ratio of the ellipse
    aspect_ratio = min(a, b) / max(a, b)
    
    # Check if the aspect ratio is within a certain range
    if aspect_ratio >= 0.7 and aspect_ratio <= 0.9:
        # Draw the ellipse on the original image
        cv2.ellipse(img, ellipse, (0, 255, 0), 2)
        
        # Display the image with the selected ellipse
        cv2.imshow('Ellipse', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
        # Save the selected ellipse as a new image
        mask = np.zeros_like(img, dtype=np.uint8)
        cv2.ellipse(mask, ellipse, (255, 255, 255), -1)
        selected_ellipse_img = cv2.bitwise_and(img, mask)
        cv2.imwrite('selected_ellipse.jpg', selected_ellipse_img)
